In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:

# Importing the necessary Libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

# Data Loading 

In [ ]:
train = "/kaggle/input/chest-xray-pneumonia/chest_xray/train"
val = "/kaggle/input/chest-xray-pneumonia//chest_xray/val"
test = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

# Data Preprocessing 

In [ ]:
input_shape = (224, 224, 3)
np.random.seed(42)
tf.random.set_seed(42)
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = datagen.flow_from_directory(
    train,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    shuffle=False
)
val_generator = datagen.flow_from_directory(
    val,
    target_size=input_shape[:2],
    batch_size=1,
    class_mode='binary',
    shuffle=False
)
test_generator = datagen.flow_from_directory(
    test,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Model Building 

In [ ]:
# Create a CNN model for anomaly detection
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training 

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

# Plotting the accuracies

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy', color='k')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='r')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Training Loss', color='k')
plt.plot(history.history['val_loss'], label='Validation Loss', color='r')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# # Model Structure 

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_structure.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.save("anamoly_detect.h5")

In [ ]:
# Predict anomaly scores for the test images
anomaly_scores = model.predict(val_generator)

# Classification of the images 

In [ ]:
# Define a threshold for anomaly detection
threshold = 0.5

# Classify the test images as normal or anomalous based on the threshold
for i, score in enumerate(anomaly_scores):
    if score < threshold:
        classification = 'Normal'
        normal_image = val_generator[i][0][0] * 255.0  # Retrieve a normally classified image
        break

for i, score in enumerate(anomaly_scores):
    if score >= threshold:
        classification = 'Anomalous'
        anomalous_image = val_generator[i][0][0] * 255.0  # Retrieve an anomalous image
        break

In [ ]:
# Display the images and their classifications
plt.subplot(1, 2, 1)
plt.imshow(normal_image.astype(np.uint8))
plt.title('Normal')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(anomalous_image.astype(np.uint8))
plt.title('Anomalous')
plt.axis('off')

plt.tight_layout()
plt.show()
